In [1]:
import pandas as pd
import re

def clean_data(df):
    # Define the regex pattern to match digitXdigit pairs or sequences containing multiple pairs
    pattern = re.compile(r'(\d+(\.\d+)?\s*[xX]\s*\d+(\.\d+)?[^\d]*?(?:\d+(\.\d+)?\s*[xX]\s*\d+(\.\d+)?[^\d]*?)*)')
    # Function to extract digitXdigit pairs from each cell value
    def extract_pairs(text):
        if isinstance(text, str):
            match = pattern.search(text)
            return match.group(0).strip() if match else None
        else:
            return None
    # Extract the digitXdigit pairs
    df.insert(7, 'size', df['physical_description'].apply(extract_pairs))
    # df["size"] = df["physical_description"].apply(extract_pairs)
    # Remove leading and trailing whitespace in column: 'size'
    df['size'] = df['size'].str.strip()
    # Created column 'size_null' from formula
    def determine_is_na(row):
        if pd.isna(row['physical_description']) or row['physical_description'].strip() == "":
            return None
        return pd.isna(row['size'])
    # Apply the function to create the 'is_na' column
    df['is_na'] = df.apply(determine_is_na, axis=1)
    # Insert the 'is_na' column at index 8
    # Note: If you have fewer than 8 columns, this will add the 'is_na' column at the end of the DataFrame
    df.insert(8, 'is_na', df.pop('is_na'))
    # Sort by column: 'is_na' (descending)
    df = df.sort_values(['is_na'], ascending=[False], key=lambda s: s.apply(str) if s.name in ['is_na'] else s)
    def get_last_value(cell):
        if isinstance(cell, str):
            parts = cell.rstrip(';').split(';')
            return parts[-1].strip()
        return None
    df['size'] = df.apply(
        lambda row: get_last_value(row['physical_description']) if row['is_na'] else row.get('size'),
        axis=1
    )
    return df

# Loaded variable 'df' from URI: /Users/lanli/data_cleaning_exp/pd_exp_prep/data_output/menu_wrangler.csv
df = pd.read_csv(r'/Users/lanli/data_cleaning_exp/pd_exp_prep/data_output/menu_wrangler.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,id,name,sponsor,event,venue,place,physical_description,size,is_na,occasion,...,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
8772,26165,NaN,ROYAL ORDER OF SCOTLAND,ANNUAL DINNER,PAT;,"GRAND HOTEL, [CINCINNATI, OH]",FOLDER; ILLUS; COL; 4.5X.6.5;,4.5X.6.5,True,ANNIVERSARY;,...,NaN,NaN,1898-09-19,Royal Order Of Scotland,NaN,NaN,NaN,complete,4,23
6438,23252,NaN,WILLARDS HOTEL,DINNER MENU,NaN,"WASHINGTON,D.C.",ILLUS. FOLDER,ILLUS. FOLDER,True,NaN,...,NaN,NaN,1864-02-12,Willards Hotel,NaN,Dollars,$,complete,4,178
7978,25202,NaN,CELTIC CLUB,FOURTHY ANNUAL BANQUET CELEBRATING THE BIRTH O...,SOC;,"STAUCH PALACE HOTEL; CONEY ISLAND, NY",BOOKLET; ILLUS; COL;,COL,True,ANNIVERSARY;,...,NaN,NaN,1907-03-18,Celtic Club,NaN,NaN,NaN,complete,12,18
7955,25171,NaN,CABARET DU CHAT NOIR,LUNCH & DINNER,COMMERCIAL,32 WEST 28 ST. NY,FOLDER; ILLUS; COL;,COL,True,DAILY,...,NaN,NaN,1900-02-03,Cabaret Du Chat Noir,NaN,Dollars,$,complete,4,313
7930,25137,NaN,NEW HAMPSHIRE CLUB,DINNER,NaN,YOUNG'S HOTEL,BROADSIDE,BROADSIDE,True,NaN,...,NaN,NaN,1880-12-08,New Hampshire Club,NaN,NaN,NaN,complete,4,31
